In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
from reportlab.pdfgen import canvas
import random

from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
pdfmetrics.registerFont(TTFont('sans-serif', 'FreeSans.ttf'))

import generating_cathegories_functions as funs
from puzzle_class import puzzle

# Rysowanie do PDF

In [43]:
def rysuj_zagadke(kategorie, clues, c, X = 30, Y = 30, box_size = 100, text_box_size = 150):
    
    K_cat = (len(kategorie)) # liczba kategorii
    k_cat = len(kategorie[0][1]) # liczba obiektów z każdej kategorii
    N_rows = K_cat-1
    width = box_size/k_cat

    cm = 20

    c.setLineWidth(2)
    for row in range(N_rows):
        c.rect( X, Y+row*box_size, text_box_size+(row+1)*box_size, box_size)
        c.rect( X+text_box_size+row*box_size, Y+row*box_size, box_size, text_box_size+(N_rows-row)*box_size)
    c.setLineWidth(1)
    for row in range(N_rows):
        for k in range(k_cat):
            c.rect( X, Y+row*box_size, text_box_size+(row+1)*box_size, k*box_size/k_cat)
            c.rect( X+text_box_size+row*box_size, Y+row*box_size, k*box_size/k_cat, text_box_size+(N_rows-row)*box_size)
    
    Xc = 30
    Yc = 830
    odstep = 0.15
    c.setFont("sans-serif", width*0.5)
    for i, clue in enumerate(clues):
        if "i3" not in clue:
            wskaz = str(kategorie[clue["K1"]][1][clue["i1"]])+" nie pasuje do "+str(kategorie[clue["K2"]][1][clue["i2"]])
        elif "i4" not in clue:
            wskaz = str(kategorie[clue["K1"]][1][clue["i1"]])+" nie pasuje ani do "+str(kategorie[clue["K2"]][1][clue["i2"]])+" ani do "+str(kategorie[clue["K2"]][1][clue["i3"]])
        else:
            wskaz = str(kategorie[clue["K1"]][1][clue["i1"]])+" nie pasuje ani do "+str(kategorie[clue["K2"]][1][clue["i2"]])+" ani do "+str(kategorie[clue["K2"]][1][clue["i3"]])+" ani do "+str(kategorie[clue["K2"]][1][clue["i4"]])
            
        c.drawString(Xc+odstep*width, Yc-width*i*0.5, str(i+1)+". "+wskaz)
    
    for i, kategoria in enumerate(kategorie):
        nazwy = [ str(k) for k in kategoria[1] ]
        if kategoria[0]=='numerical':
            nazwy = [ k[:-2] if k.endswith(".0") else k for k in nazwy ]
            if "@" in kategoria[3]:
                a = kategoria[3].split("@")
                nazwy = [ k.join(a) for k in nazwy ]
        
        miejsce = i+1
        odstep = 0.15
        for i, name in enumerate(nazwy):
            if len(name) > 7+(k_cat-3)*3:
                c.setFont("sans-serif", width*(7+(k_cat-3)*3)/len(name))
            else:
                c.setFont("sans-serif", width)
            # rysowanie poziome:
            if miejsce==1:
                c.drawString(X+odstep*width, Y+(K_cat-1)*box_size-(i+1)*width+odstep*width, name)
            elif miejsce!=2:
                c.drawString(X+odstep*width, Y+(miejsce-2)*box_size-(i+1)*width+odstep*width, name)
            c.saveState()
            # rysowanie pionowe:
            c.rotate( 90 )
            if miejsce==2:
                c.drawString(Y+(K_cat-1)*box_size+odstep*width, -X-text_box_size-(i+1)*width+odstep*width, name)
            elif miejsce!=1:
                c.drawString(Y+(K_cat-1)*box_size+odstep*width, -X-text_box_size-(miejsce-2)*box_size-(i+1)*width+odstep*width, name)

            c.restoreState()



In [45]:
kategorie = funs.losuj_kategorie(4, 5, 3, random.randint(100,100000))

puzzle1 = puzzle(4,5)
for i in range(40):
    puzzle1.add_clue1()
    puzzle1.use_clue1(i)
    puzzle1.grid_concile()
    if puzzle1.is_grid_completed() or puzzle1.is_grid_contradictory():
        break

if puzzle1.is_grid_contradictory():
    print("Wyszło sprzeczne!")
        
c = canvas.Canvas("rotate.pdf")
rysuj_zagadke(kategorie, puzzle1.clues, c)
c.showPage()
c.save()

In [5]:
kategorie

[('cathegorical', ['tarcza', 'włócznia', 'przyłbica', 'topór', 'napierśnik']),
 ('cathegorical',
  ['Jankowski', 'Grabowski', 'Kwiatkowski', 'Wiśniewski', 'Mazur']),
 ('numerical',
  [2.5, 5.0, 10.0, 12.5, 15.0],
  {'x=y*2', 'x=y+10.0', 'x=y+2.5', 'x=y+5.0', 'x=y+7.5'},
  'zysk/strata'),
 ('numerical',
  [3000.0, 3100.0, 3200.0, 3300.0, 3400.0],
  {'x=y+100', 'x=y+200', 'x=y+300'},
  '@ rok/lata')]

In [3]:
puzzle1 = puzzle(4,5)

for i in range(40):
    puzzle1.add_clue1()
    puzzle1.use_clue1(i)
    puzzle1.grid_concile()
    
    if puzzle1.is_grid_completed() or puzzle1.is_grid_contradictory():
        print(puzzle1.is_grid_completed(), puzzle1.is_grid_contradictory())
        break
puzzle1.print_grid()
print(len(puzzle1.clues))

True False
XXOXX OXXXX XOXXX 
XXXOX XXXXO XXXXO 
XOXXX XOXXX XXOXX 
XXXXO XXOXX OXXXX 
OXXXX XXXOX XXXOX 

XXXXO XXOXX 
XXOXX OXXXX 
XOXXX XOXXX 
OXXXX XXXOX 
XXXOX XXXXO 

XXOXX 
XOXXX 
XXXXO 
OXXXX 
XXXOX 

26


# Inne

In [67]:
def fonts(canvas):
    from reportlab.lib.units import inch
    for font in canvas.getAvailableFonts():
        print(font)
fonts(c)

Courier
Courier-Bold
Courier-BoldOblique
Courier-Oblique
Helvetica
Helvetica-Bold
Helvetica-BoldOblique
Helvetica-Oblique
Symbol
Times-Bold
Times-BoldItalic
Times-Italic
Times-Roman
ZapfDingbats
sans-serif


# Losowanie wszystkiego

In [9]:
list(np.random.choice([1,2,3,4], 4, replace=True)) 

[3, 2, 4, 2]

In [16]:
funs.losuj_kategorie(5, 5, 3, np.random.randint(1002))

['special_names' 'items' 'names']
special_names
1 items
special_names
1 names
special_names
1 special_names


[('cathegorical', ['metanol', 'metan', 'kwa mrówkowy', 'gliceryna', 'cukier']),
 ('cathegorical', ['Zosia', 'Janek', 'Ryś', 'Kubuś', 'Tomek']),
 ('cathegorical',
  ['łamigłówki', 'obrazki logiczne', 'architekty', 'wielokropki', 'sudoku']),
 ('numerical',
  [1000.0, 2000.0, 3000.0, 4000.0, 5000.0],
  {'x=y*2', 'x=y+1000', 'x=y+2000', 'x=y+3000'},
  '@ rok/lata'),
 ('numerical',
  [4.0, 8.0, 12.0, 20.0, 28.0],
  {'x=y+16.0', 'x=y+4.0', 'x=y+8.0'},
  'zysk/strata')]

# Klasa wskazowka

In [23]:
class wskazowka:
    """
    Klasa pomocniczna do zagadki.
    """
    __slots__ = ["typ", "podtyp", "text"]
    def __init__(self, typ, podtyp, seed):
        self.typ = typ
        self.podtyp = podtyp
        self.text = ""
    def __repr__(self):
        return "Typ: "+str(self.typ)+"/"+str(self.podtyp)+"\nWskazówka: "+self.text

In [24]:
w1 = wskazowka(1, 1, 0)

In [25]:
print(w1)

Typ: 1/1
Wskazówka: 


In [26]:
w1.text

''

In [27]:
w1.__slots__

['typ', 'podtyp', 'text']

In [45]:
class zagadka:
    """
    Główna klasa konkretnej zagadki zawierającej K-kategorii po k-obiektów w każdej.
    Klasa zawiera listę wskazówek(clues), opis kategorii(cathegories) 
    oraz planszę do rozwiazywania(boxes).
    """
    __slots__ = ["K", "k", "status", "diff", "diff_expected", "clues", "cathegories", "boxes"]
    
    def __init__(self, K, k, diff_expected):
        self.K = K
        self.k = k
        self.status = "untouched"
        self.diff = "unknown"
        self.diff_expected = diff_expected
        self.clues = []
        self.cathegories = []
        self.boxes = []
        for i in range(K-1):
            self.boxes.append([])
            for j in range(i,K-1):
                self.boxes[i].append(np.zeros((k,k)))
    
    def __repr__(self):
        return ", ".join(self.clues)

In [46]:
z = zagadka(5, 4, 2)
print(z)

In [49]:
z.boxes[0][4]

IndexError: list index out of range

In [36]:
K = 5
k = 5
kat = funs.losuj_kategorie(K, k, 3, np.random.randint(1002))
box = np.zeros((k,k))


In [37]:
box

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])